# Cheatsheet: Scenarios
This notebook provides quick examples of methods for using `Scenario` objects to add data or other content to your [EDSL](https://docs.expectedparrot.com/) survey questions. Scenarios allow you to efficiently administer multiple versions of questions at once, which can be useful in conducting experiments and labeling/exploration tasks where you want to answer the same questions about many different things, such as every piece of data in a dataset, or a collection of texts, websites, images or other content.

In the examples below we show how to:

* Inspect an example scenario
* Use a scenario in a question
* Create scenarios
* Combine scenarios
* Replicate scenarios
* Rename scenario keys
* Sample scenarios
* Select and drop scenarios
* Slice & chunk scenarios
* Create scenarios for texts, PDFs, CSVs, URLs, HTML, images, dictionaries & lists
* Add metadata to scenarios
* Generate code for recreating scenarios

## What is EDSL?
[EDSL](https://github.com/expectedparrot/edsl) is an open-source Python library for simulating surveys, experiments and other research with AI agents and large language models. It is buily by [Expected Parrot] Please see our [documentation page](https://docs.expectedparrot.com/) for information and tutorials on getting started, and more details on [methods for working with scenarios](https://docs.expectedparrot.com/en/latest/scenarios.html) that are explored in this notebook.

## How to use this notebook
The examples in this notebook can be rerun or modified to use your own questions and data or content. We also post each example at the [Coop](https://www.expectedparrot.com/explore)--*a new platform for creating, storing and sharing LLM-based research using EDSL*--where you can interact with them directly as well. [Learn more about using the Coop](https://docs.expectedparrot.com/en/latest/coop.html) to conduct research.

## Technical setup
Before running the code below, ensure that you have (1) [installed](https://docs.expectedparrot.com/en/latest/installation.html) the EDSL library and (2) created a [Coop account](https://www.expectedparrot.com/login) to activate [remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) or stored your own [API keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for language models that you want to use with EDSL.

## Importing the tools
We start by importing the relevant tools:

In [1]:
from edsl import Scenario, ScenarioList

## Inspecting an example
A `Scenario` contains a dictionary of keys and values representing data or content to be added to (inserted in) the `question_text` field of a `Question` object. We can call the `example()` method to inspect an example scenario:

In [2]:
example_scenario = Scenario.example()
example_scenario

Scenario({'persona': 'A reseacher studying whether LLMs can be used to generate surveys.'})

We can also see an example `ScenarioList`, which is a dictionary containing a list of scenarios:

In [3]:
example_scenariolist = ScenarioList.example()
example_scenariolist

ScenarioList([Scenario({'persona': 'A reseacher studying whether LLMs can be used to generate surveys.'}), Scenario({'persona': 'A reseacher studying whether LLMs can be used to generate surveys.'})])

## Using a Scenario
To use a scenario:
1. Create a `Question` (or a `Survey` of multiple questions) that includes a `{{ placeholder }}` in the `question_text` for each scenario key.
2. Call the `by()` method on the question (or survey) and pass it the scenario (or list of scenarios). A new version of a question is automatically created with each scenario value.
3. Call the `run()` method on the question (or survey) to send it to a language model. Responses are returned in a formatted dataset of results that includes details on all components.

Here we construct a survey of questions using the example scenario from above, send it to a language model and inspect the results. If [remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) is activated, the results are also automatically posted to the Coop:

In [4]:
# Import question types
from edsl import QuestionFreeText, QuestionList, Survey

# Create questions in the relevant templates with placeholders for content to be inserted
q1 = QuestionFreeText(
    question_name = "background",
    question_text = "Draft a sample bio for this researcher: {{ persona }}",
)
q2 = QuestionList(
    question_name = "interests",
    question_text = "Identify some potential interests of this researcher: {{ persona }}",
)

# Combine questions into a survey to administer them together
survey = Survey(questions=[q1, q2])

# Run the survey with the scenarios to generate a dataset of results
results = survey.by(example_scenario).run()

SyntaxError: invalid decimal literal (InferenceServicesCollection.py, line 37)

We can analyze the results at the Coop or using built-in [methods for working with results](https://docs.expectedparrot.com/en/latest/results.html) at your workspace:

In [ ]:
# Print a table of selected components of the results
results.select("persona", "background", "interests").print(format="rich")

## Creating a Scenario
We create a scenario by passing a dictionary to a `Scenario` object:

In [ ]:
weather_scenario = Scenario({"weather": "sunny"})
weather_scenario

## Creating a ScenarioList
It can be useful to create a set of scenarios all at once. This can be done by constructing a list of `Scenario` objects or a `ScenarioList`. Compare a list of `Scenario` objects:

In [ ]:
weather_scenarios = [
    Scenario({"weather": w}) for w in ["sunny", "cloudy", "rainy", "snowy"]
]
weather_scenarios

Alternatively, we can create a `ScenarioList` which has a key `scenarios` and a list of scenarios as the values:

In [ ]:
example_scenariolist = ScenarioList.example()
example_scenariolist

In [ ]:
weather_scenariolist = ScenarioList(
    [Scenario({"weather": w}) for w in ["sunny", "cloudy", "rainy", "snowy"]]
)
weather_scenariolist

## Combining scenarios
We can add scenarios together to create a single new scenario with an extended dictionary:

In [ ]:
scenario1 = Scenario({"food": "apple"})
scenario2 = Scenario({"drink": "juice"})

snack_scenario = scenario1 + scenario2
snack_scenario

## Replicating scenarios
We can replicate a scenario to create a `ScenarioList`:

In [ ]:
personas_scenariolist = Scenario.example().replicate(n=3)
personas_scenariolist

## Renaming scenarios
We can call the `rename()` method to rename the fields (keys) of a `Scenario`:

In [ ]:
role_scenario = Scenario.example().rename({"persona": "role"})
role_scenario

The method can also be called on a `ScenarioList`:

In [ ]:
scenariolist = ScenarioList(
    [
        Scenario({"name": "Apostolos"}),
        Scenario({"name": "John"}),
        Scenario({"name": "Robin"}),
    ]
)

renamed_scenariolist = scenariolist.rename({"name": "first_name"})
renamed_scenariolist

## Sampling
We can call the `sample()` method to take a sample from a `ScenarioList`:

In [ ]:
weather_scenariolist = ScenarioList(
    [Scenario({"weather": w}) for w in ["sunny", "cloudy", "rainy", "snowy"]]
)

sample = weather_scenariolist.sample(n=2)
sample

## Selecting and dropping scenarios
We can call the `select()` and `drop()` methods on a `ScenarioList` to include and exclude specified fields from the scenarios:

In [ ]:
snacks_scenariolist = ScenarioList(
    [
        Scenario({"food": "apple", "drink": "water"}),
        Scenario({"food": "banana", "drink": "milk"}),
    ]
)

food_scenariolist = snacks_scenariolist.select("food")
food_scenariolist

In [ ]:
drink_scenariolist = snacks_scenariolist.drop("food")
drink_scenariolist

## Adding metadata to scenarios
Note that we can create fields in scenarios without including them in the `question_text`. This will cause the fields to be present in the `Results` dataset, which can be useful for adding metadata to your questions and results. [See more examples here](https://docs.expectedparrot.com/en/latest/notebooks/adding_metadata.html).

Example usage:

In [ ]:
songs = [
    ["1999", "Prince", "pop"],
    ["1979", "The Smashing Pumpkins", "alt"],
    ["1901", "Phoenix", "indie"],
]
metadata_scenarios = [
    Scenario({"title": t, "musician": m, "genre": g}) for [t, m, g] in songs
]
metadata_scenarios

In [ ]:
q = QuestionFreeText(
    question_name="song",
    question_text="What is this song about: {{ title }}",  # optionally omitting other fields in the scenarios
)

results = q.by(metadata_scenarios).run()
results.select("scenario.*", "song").print(
    format="rich"
)  # all scenario fields will be present

Note that it does not matter if we use a list of `Scenario` objects or a `ScenarioList` with the same data--the scenarios are added to the survey in the same way when it is run:

In [ ]:
songs = [
    ["1999", "Prince", "pop"],
    ["1979", "The Smashing Pumpkins", "alt"],
    ["1901", "Phoenix", "indie"],
]
metadata_scenarios = ScenarioList(
    [Scenario({"title": t, "musician": m, "genre": g}) for [t, m, g] in songs]
)
metadata_scenarios

In [ ]:
q = QuestionFreeText(
    question_name="song",
    question_text="What is this song about: {{ title }}",  # optionally omitting other fields in the scenarios
)

results = q.by(metadata_scenarios).run()
results.select("scenario.*", "song").print(
    format="rich"
)  # all scenario fields will be present

## Chunking text
We can use the `chunk()` method to turn a `Scenario` into a `ScenarioList` with specified slice/chunk sizes based on `num_words` or `num_lines`. Note that the field `_chunk` is created automatically, and `_original` is added if optional parameter `include_original` is used:

In [ ]:
my_haiku = """
This is a long text. 
Pages and pages, oh my!
I need to chunk it.
"""

text_scenario = Scenario({"my_text": my_haiku})

word_chunks_scenariolist = text_scenario.chunk(
    "my_text",
    num_words=5,  # use num_words or num_lines but not both
    include_original=True,  # optional
    hash_original=True,  # optional
)
word_chunks_scenariolist

In [ ]:
line_chunks_scenariolist = text_scenario.chunk("my_text", num_lines=1)
line_chunks_scenariolist

## Tallying scenario values
We can call the `tally()` method on a `ScenarioList` to tally numeric values for a specified key. It returns a dictionary with keys representing the number of each `Scenario` in the `ScenarioList` and values representing the tally of the key that was specified:

In [ ]:
numeric_scenariolist = ScenarioList(
    [Scenario({"a": 1, "b": 1}), Scenario({"a": 1, "b": 2})]
)

tallied_scenariolist = numeric_scenariolist.tally("b")
tallied_scenariolist

## Expanding scenarios
We can call the `expand()` method on a `ScenarioList` to expand it by a specified field. For example, if the values of a scenario key are a list we can pass that key to the method to generate a `Scenario` for each item in the list:

## Mutating scenarios
We can call the `mutate()` method on a `ScenarioList` to add a key/value to each `Scenario` based on a logical expression:

In [ ]:
scenariolist = ScenarioList([Scenario({"a": 1, "b": 1}), Scenario({"a": 1, "b": 2})])

mutated_scenariolist = scenariolist.mutate("c = a + b")
mutated_scenariolist

## Ordering scenarios
We can call the `order_by()` method on a `ScenarioList` to order the scenarios by a field:

In [ ]:
unordered_scenariolist = ScenarioList(
    [Scenario({"a": 1, "b": 1}), Scenario({"a": 1, "b": 2})]
)

ordered_scenariolist = unordered_scenariolist.order_by("b")
ordered_scenariolist

## Filtering scenarios
We can call the `filter()` method on a `ScenarioList` to filer scenarios based on a conditional expression.

In [ ]:
unfiltered_scenariolist = ScenarioList(
    [Scenario({"a": 1, "b": 1}), Scenario({"a": 1, "b": 2})]
)

filtered_scenariolist = unfiltered_scenariolist.filter("b == 2")
filtered_scenariolist

## Create scenarios from a list
We can call the `from_list()` method to create a `ScenarioList` from a list of values and a specified key:

In [ ]:
my_list = ["Apostolos", "John", "Robin"]

scenariolist = ScenarioList.from_list("name", my_list)
scenariolist

## Adding a list of values to individual scenarios
We can call the `add_list()` method to add values to individual scenarios in a `ScenarioList`:

In [ ]:
scenariolist = ScenarioList(
    [Scenario({"weather": "sunny"}), Scenario({"weather": "rainy"})]
)

added_scenariolist = scenariolist.add_list("preference", ["high", "low"])
added_scenariolist

## Adding values to all scenarios
We can call the `add_value()` to add a value to all scenarios in a `ScenarioList`:

In [ ]:
scenariolist = ScenarioList(
    [
        Scenario({"name": "Apostolos"}),
        Scenario({"name": "John"}),
        Scenario({"name": "Robin"}),
    ]
)

added_scenariolist = scenariolist.add_value("company", "Expected Parrot")
added_scenariolist

## Creating scenarios from a pandas DataFrame
We can call the `from_pandas()` method to create a `ScenarioList` from a pandas DataFrame:

In [ ]:
import pandas as pd

df = pd.DataFrame(
    {
        "name": ["Apostolos", "John", "Robin"],
        "location": ["New York", "Cambridge", "Cambridge"],
    }
)

scenariolist = ScenarioList.from_pandas(df)
scenariolist

## Creating scenarios from a CSV
We can call the `from_csv()` method to create a `ScenarioList` from a CSV:

In [ ]:
scenariolist = ScenarioList.from_csv("example.csv")
scenariolist

## Turn a ScenarioList into a dictionary
We can call the `to_dict()` method to turn a `ScenarioList` into a dictionary:

In [ ]:
scenariolist = ScenarioList(
    [
        Scenario({"name": "Apostolos"}),
        Scenario({"name": "John"}),
        Scenario({"name": "Robin"}),
    ]
)

dict_scenariolist = scenariolist.to_dict()
dict_scenariolist

## Create a ScenarioList from a dictionary
We can call the `from_dict()` method to create a `ScenarioList` from a dictionary. Note that the dictionary must contain a key "scenarios":

## Turning PDF pages into scenarios
We can call the `from_pdf()` method to turn the pages of a PDF or doc into a `ScenarioList`. Here we use it for John's paper <i>"Large Language Models as Simulated Economic Agents: What Can We Learn from Homo Silicus?"</i> ([link to paper](https://arxiv.org/pdf/2301.07543)). Note that the keys `filename`, `page` and `text` are automatically specified, so the `question_text` placeholder that we use for the scenarios must be `{{ text }}`:

In [ ]:
pdf_pages_scenariolist = ScenarioList.from_pdf("homo_silicus.pdf")
pdf_pages_scenariolist[0:2]  # inspecting the first couple pages as scenarios

Example usage:

## Turning PDF pages into scenarios
We can call the `from_pdf()` method to turn the pages of a PDF or doc into a `ScenarioList`. Here we use it for John's paper <i>"Large Language Models as Simulated Economic Agents: What Can We Learn from Homo Silicus?"</i> ([link to paper](https://arxiv.org/pdf/2301.07543)). Note that the keys `filename`, `page` and `text` are automatically specified, so the `question_text` placeholder that we use for the scenarios must be `{{ text }}`:

In [ ]:
homo_silicus_scenariolist = ScenarioList.from_pdf("homo_silicus.pdf")

Here we inspect a couple pages:

In [ ]:
homo_silicus_scenariolist["scenarios"][0:2]

Example usage--note that we can [sort results](https://docs.expectedparrot.com/en/latest/results.html#sorting-results) by any component, [filter results](https://docs.expectedparrot.com/en/latest/notebooks/docs_questions.html#Filtering-results) using conditional expressions, and also [limit how many results to display](https://docs.expectedparrot.com/en/latest/results.html#limiting-results):

In [ ]:
q = QuestionFreeText(
    question_name="summarize", question_text="Summarize this page: {{ text }}"
)
results = q.by(homo_silicus_scenariolist).run()

In [ ]:
(
    results.sort_by("page")
    .filter("page > 1")
    .select("page", "summarize")
    .print(format="rich", max_rows=3)
)

## Using images as scenarios
We can call the `from_image()` method to create a scenario for an image. Here we use it for Figure 1 in the <i>Home Silicus</i> paper.

Note that this method must be used with a vision model (e.g., GPT-4o) and does not require the use of a `{{ placeholder }}` in the question text. The scenario keys `file_path` and `encoded_image` are generated automatically:

In [ ]:
from edsl import Model

model = Model("gpt-4o")

In [ ]:
image_scenario = Scenario.from_image("homo_silicus_figure1.png")

In [ ]:
image_scenario.keys()

Example usage:

In [ ]:
q = QuestionFreeText(
    question_name="figure",
    question_text="Explain the graphic on this page.",  # no scenario placeholder
)

results = q.by(image_scenario).by(model).run()
results.select("figure").print(format="rich")

In [ ]:
scenariolist = ScenarioList([Scenario({"a": 1, "b": [1, 2, 3]})])

expanded_scenarios = scenariolist.expand("b")
expanded_scenarios

## Generating code for scenarios
We can call the `code()` method to generate the code for producing scenarios:

In [ ]:
scenariolist = ScenarioList.example()

scenariolist_code = scenariolist.code()
scenariolist_code

In [ ]:
from edsl.scenarios.Scenario import Scenario
from edsl.scenarios.ScenarioList import ScenarioList

scenario_0 = Scenario(
    {"persona": "A reseacher studying whether LLMs can be used to generate surveys."}
)
scenario_1 = Scenario(
    {"persona": "A reseacher studying whether LLMs can be used to generate surveys."}
)
scenarios = ScenarioList([scenario_0, scenario_1])

## Converting a `ScenarioList` into an `AgentList`
We can call the `to_agent_list()` method to convert a `ScenarioList` into an `AgentList`. Note that agent `traits` cannot include a "name" key as `agent_name` is a separate optional field of `Agent` objects:

In [ ]:
from edsl import AgentList

scenariolist = ScenarioList(
    [
        Scenario({"first_name": "Apostolos", "location": "New York"}),
        Scenario({"first_name": "John", "location": "Cambridge"}),
        Scenario({"first_name": "Robin", "location": "Cambridge"}),
    ]
)

agentlist = scenariolist.to_agent_list()
agentlist

(Note that scenarios function similarly to `traits` dictionaries that we pass to AI `Agents` that we can use to answer survey questions. [Learn more about designing AI agents](https://docs.expectedparrot.com/en/latest/agents.html) for simulating surveys and experiments.)